# Kalman Filter

In [1]:
import sys
# Add the symgp folder path to the sys.path list
module_path = r'/Users/jaduol/Documents/Uni (original)/Part II/IIB/MEng Project/'
if module_path not in sys.path:
    sys.path.append(module_path)

from symgp import SuperMatSymbol, utils, MVG, Variable, SuperDiagMat, Kernel, Covariance, Constant, Mean
from sympy import symbols, ZeroMatrix, Identity
from IPython.display import display, Math, Latex, clear_output

In [2]:
# Shapes
m, n, t = symbols('m n t')

## 1. Variables

In [3]:
# Variables
x_prev, x_t, y_prev, y_t, b_t, d_t, w_t, e_t = utils.variables('x_{t-1} x_t y_{1:t-1} y_t b_t d_t w_t e_t', 
                                                                [m, m, (n,t-1), n, m, n, m, n])

## 2. Constant Parameters

In [4]:
# Constant parameters
A, C = utils.constants('A C',[(m,m), (n,m)])                                  
Q = Covariance(w_t, name='Q')
R = Covariance(e_t, name='R')

## 3. p(x_(t-1)|y_(1:t-1))

In [5]:
# p(x_(t-1)|y_(1:t-1))
f_prev = Mean(x_prev, name='m_{t-1|t-1}') 
F_prev = Covariance(x_prev, name='P_{t-1|t-1}') 
p_xprev = MVG([x_prev], mean=f_prev, cov=F_prev, cond_vars=[y_prev])

print("p_xprev:")
display(Latex(utils.matLatex(p_xprev)))

p_xprev:


<IPython.core.display.Latex object>

## 4. p(x_t|x_(t-1))

In [6]:
p_xt = MVG([x_t], mean=A*x_prev + b_t, cov=Q, cond_vars=[x_prev])

print("p_xt:")
display(Latex(utils.matLatex(p_xt)))

p_xt:


<IPython.core.display.Latex object>

## 5. p(x_t,x_(t-1)|y_(1:t-1))

In [7]:
p_xt_xprev = p_xt*p_xprev

print("p_xt_xprev:")
display(Latex(utils.matLatex(p_xt_xprev)))

p_xt_xprev:


<IPython.core.display.Latex object>

In [8]:
print(p_xt_xprev.covar.blockform)
print(p_xt_xprev.covar.to_full_expr())

[[S_{x_t,x_t}, S_{x_t,x_{t-1}}], [S_{x_{t-1},x_t}, S_{x_{t-1},x_{t-1}}]]
[[Q + A*P_{t-1|t-1}*A', A*P_{t-1|t-1}], [P_{t-1|t-1}*A', P_{t-1|t-1}]]


## 6. p(x_t|y_(1:t-1))

In [9]:
p_xt_predict = p_xt_xprev.marginalise([x_prev])

print("p_xt_predict:")
display(Latex(utils.matLatex(p_xt_predict)))

p_xt_predict:


<IPython.core.display.Latex object>

In [10]:
print(utils.matLatex(p_xt_predict))

\begin{align*}
p\left(\mathbf{x_t}|\mathbf{y_{1:t-1}}\right)&= \mathcal{N}\left(\mathbf{x_t};\mathbf{m}_{\mathbf{x_t}|\mathbf{y_{1:t-1}}},\mathbf{\Sigma}_{\mathbf{x_t}|\mathbf{y_{1:t-1}}}\right)\\
\mathbf{m}_{\mathbf{x_t}|\mathbf{y_{1:t-1}}} &= \mathbf{A} \mathbf{m_{t-1|t-1}} + \mathbf{b_t}\\
\mathbf{\Sigma}_{\mathbf{x_t}|\mathbf{y_{1:t-1}}} &= \mathbf{Q} + \mathbf{A} \mathbf{P_{t-1|t-1}} \mathbf{A}^T\\
\end{align*}


## 7. p(y_t|x_t)

In [11]:
p_yt = MVG([y_t], mean=C*x_t + d_t, cov=R, cond_vars=[x_t])

print("p_yt:")
display(Latex(utils.matLatex(p_yt)))

p_yt:


<IPython.core.display.Latex object>

## 8. p(y_t ,x_t|y_(1:t-1))

In [12]:
p_yt_xt = p_yt*p_xt_predict

print("p_yt_xt:")
display(Latex(utils.matLatex(p_yt_xt)))

p_yt_xt:


<IPython.core.display.Latex object>

In [13]:
print(p_yt_xt.covar.blockform[0][0])

S_{y_t,y_t}


## 9. p(x_t|y_(1:t))

In [14]:
p_xt_update = p_yt_xt.condition([y_t])

print("p_xt_update:")
display(Latex(utils.matLatex(p_xt_update)))

p_xt_update:


<IPython.core.display.Latex object>

In [15]:
#from sympy import srepr
#print(srepr(p_xt_update.mean.to_full_expr()))

In [16]:
from sympy import Matrix, Identity
import numpy as np

d = {'A': np.array([[1, 0, 1, 0],[0, 1, 0, 1],[0, 0, 1, 0],[0, 0, 0, 1]],dtype=np.float32),
     'C': np.array([[1, 0, 0, 0],[0, 1, 0, 0]], dtype=np.float32),
     'Q': 0.001*np.eye(4),#Matrix(Identity(4)),
     'R': np.eye(2),#Matrix(Identity(2)),
     'm_{t-1|t-1}': np.array([[8],[10],[1],[0]],dtype=np.float32),
     'P_{t-1|t-1}': np.eye(4),#Matrix(Identity(4)),
     'b_t': np.array([[0],[0],[0],[0]],dtype=np.float32),
     'd_t': np.array([[0],[0]],dtype=np.float32)}


In [17]:
def ldsSample(A, C, Q, R, initmu, T):
    """
        Simulates a run of a linear dynamical system
    """
    
    A, C = np.array(A.tolist(), dtype=np.float32), np.array(C.tolist(), dtype=np.float32)
    Q, R = np.array(Q.tolist(), dtype=np.float32), np.array(R.tolist(), dtype=np.float32)
    initmu = np.array(initmu.tolist(), dtype=np.float32).reshape((-1,))
    
    ss = A.shape[0]
    os = C.shape[0]
    
    x = np.zeros((ss,T))
    y = np.zeros((os,T))
    
    x[:,0] = np.dot(A,initmu) + np.random.multivariate_normal(np.zeros(ss),Q)
    y[:,0] = np.dot(C,x[:,0]) + np.random.multivariate_normal(np.zeros(os),R)
    
    for t in range(1,T):
        x[:,t] = np.dot(A,x[:,t-1]) + np.random.multivariate_normal(np.zeros(ss),Q)
        y[:,t] = np.dot(C,x[:,t-1]) + np.random.multivariate_normal(np.zeros(os),R)
    
    return x,y


In [18]:
#T = 15
#x, y = ldsSample(d['A'], d['C'], d['Q'], d['R'], d['m_{t-1|t-1}'], T)

#d['y_t'] = Matrix(y[:,0].reshape((-1,1)))

In [19]:
def kalmanFilter(y, A, C, Q, R, init_m, init_V, debug=False):
    
    A, C = np.array(A.tolist(), dtype=np.float32), np.array(C.tolist(), dtype=np.float32)
    Q, R = np.array(Q.tolist(), dtype=np.float32), np.array(R.tolist(), dtype=np.float32)
    init_m = np.array(init_m.tolist(), dtype=np.float32).reshape((-1,))
    init_V = np.array(init_V.tolist(), dtype=np.float32)
    
    os, T = y.shape   #Observations shape 
    ss = A.shape[0]   #State space size
    
    m = np.zeros((ss, T))
    mpred = np.zeros((ss, T))
    V = np.zeros((ss, ss, T))
    Vpred = np.zeros((ss, ss, T))
    
    for t in range(T):
        if t==0:
            d['m_{t-1|t-1}'] = init_m
            d['P_{t-1|t-1}'] = init_V
        else:
            d['m_{t-1|t-1}'] = m[:,t-1]
            d['P_{t-1|t-1}'] = V[:,:,t-1]
        
        d['y_t'] = Matrix(y[:,t].reshape((-1,1)))
        
        if debug:
            print("t: ", t)
            print("d['m_{t-1|t-1}']: ",d['m_{t-1|t-1}'])
            print("d['P_{t-1|t-1}']: ",d['P_{t-1|t-1}'])
            print("d['y_t']: ", d['y_t'])
        
    
        m[:,t] =  utils.search_and_replace_num(p_xt_update.mean.to_full_expr(), d, debug)
        V[:,:,t] = utils.search_and_replace_num(p_xt_update.covar.to_full_expr(), d, debug)
        mpred[:,t] = utils.search_and_replace_num(p_xt_predict.mean.to_full_expr(), d, debug)
        Vpred[:,:,t] = utils.search_and_replace_num(p_xt_predict.covar.to_full_expr(), d, debug)
                
    
    return m, V, mpred, Vpred

In [20]:
#mfilt, Vfilt, mpred, Vpred = kalmanFilter(y, d['A'], d['C'], d['Q'], d['R'], d['m_{t-1|t-1}'], d['P_{t-1|t-1}'])

In [21]:
#Configure the matplotlib backend as plotting inline
"""%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches

#Enables latex
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica'],'size':'20'})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)"""

def plotGaussianEllipse(ax, m, V, ellipsecolor, markercolor,label=None,t=None):
    """
        Plots an ellipse representing the covariance matrix of a Gaussian.
        
        Args:
            ax - The Axes object on which to plot the ellipses
            m - The centre of the ellipse
            V - Covariance matrix
            ellipsecolor - Ellipse color border colour
            markercolor - Centre marker colour
          (Optional)
            label - label for the plot containing this point
            t - point at which to add new data
    """
    
    s = 4.605  #90% confidence interval
    eigVals, eigVecs = np.linalg.eig(V)
        
    #Sort eigvalues to make sure that large eigval is first
    idx = eigVals.argsort()[::-1]
    eigVals = eigVals[idx]
    eigVecs = eigVecs[:,idx]
        
    #Add ellipse to axes
    border = patches.Arc(xy=m,width = 2*(s*eigVals[0])**0.5,height = 2*(s*eigVals[1])**0.5,
                        angle=np.degrees(np.arctan2(eigVecs[1,0],eigVecs[0,0])), \
                         linewidth=0.5, linestyle='--',color=ellipsecolor,fill=False, zorder=2)
    ax.add_patch(border)
    
    #Either plot points that are joined by a line or plot them individually
    if t != None:
        data = ax.lines[-1].get_data()
        if t >= len(data[0]):
            new_xdata = np.append(data[0],m[0])
            new_ydata = np.append(data[1],m[1])
            ax.lines[-1].set_data((new_xdata, new_ydata))
        else:
            data[0][t] = m[0]
            data[1][t] = m[1]
            ax.lines[-1].set_data((data[0],data[1]))        
    else:   
        ax.plot(m[0],m[1],'x-',ms=10,mew=2,c=markercolor,label=label)
    
def plotLine(ax, a, b, color):
    """
        Plots a line between two points
    """
    ax.plot([a[0], b[0]],[a[1], b[1]], color=color,linestyle='solid')

In [22]:
"""#Set up figure
fig = plt.figure()
fig.set_size_inches(20,7)
ax = fig.add_subplot(111)
plt.axis('equal')

#Plot observations and true data
ax.set_xlim(left=5,right=25)
ax.set_ylim(bottom=5,top=15)
ax.plot(y[0,:],y[1,:],'ko',label=r'Obs')  
ax.plot(x[0,:],x[1,:],'g-s',label=r'True')
ax.set_xlabel('$x_{1}$')
ax.set_ylabel('$x_{2}$')
fig.tight_layout()

runTillEnd = False

#Plot Kalman filter estimate
for t in range(mfilt.shape[1]):
    if not runTillEnd:
        if input("Press enter to continue (Predict). Type 'rte' to run simulation to end.") == "rte":
            runTillEnd = True
            
    clear_output(wait=True)
    
    #Plot prediction
    plotGaussianEllipse(ax, mpred[:,t], Vpred[:,:,t], \
                    ellipsecolor='green', markercolor='green')
    display(plt.gcf())
    
    if not runTillEnd:
        if input("Press enter to continue (Measurement). Type 'rte' to run simulation to end.") == "rte":
            runTillEnd = True
            
    clear_output(wait=True)
    
    #Remove last prediction ellipse
    del ax.patches[-1]
    del ax.lines[-1]
            
    #Plot update
    if t == 0:
        plotGaussianEllipse(ax, mfilt[:2,t],Vfilt[:2,:2,t],\
                    ellipsecolor='blue', markercolor='red',label=r'Filter')
        ax.legend()
    else:
        plotGaussianEllipse(ax, mfilt[:2,t],Vfilt[:2,:2,t],\
                    ellipsecolor='blue', markercolor='red',t=t)
    display(plt.gcf())

clear_output(wait=True)"""

'#Set up figure\nfig = plt.figure()\nfig.set_size_inches(20,7)\nax = fig.add_subplot(111)\nplt.axis(\'equal\')\n\n#Plot observations and true data\nax.set_xlim(left=5,right=25)\nax.set_ylim(bottom=5,top=15)\nax.plot(y[0,:],y[1,:],\'ko\',label=r\'Obs\')  \nax.plot(x[0,:],x[1,:],\'g-s\',label=r\'True\')\nax.set_xlabel(\'$x_{1}$\')\nax.set_ylabel(\'$x_{2}$\')\nfig.tight_layout()\n\nrunTillEnd = False\n\n#Plot Kalman filter estimate\nfor t in range(mfilt.shape[1]):\n    if not runTillEnd:\n        if input("Press enter to continue (Predict). Type \'rte\' to run simulation to end.") == "rte":\n            runTillEnd = True\n            \n    clear_output(wait=True)\n    \n    #Plot prediction\n    plotGaussianEllipse(ax, mpred[:,t], Vpred[:,:,t],                     ellipsecolor=\'green\', markercolor=\'green\')\n    display(plt.gcf())\n    \n    if not runTillEnd:\n        if input("Press enter to continue (Measurement). Type \'rte\' to run simulation to end.") == "rte":\n            runTi

In [23]:
#utils.display_expr_tree(p_xt_update.covar.to_full_expr())
#print(utils.getMaxDepth(p_xt_update.covar.to_full_expr()))
#for k,v in utils.getExprsAtDepth(p_xt_update.covar.to_full_expr(),range(8+1),debug=False).items():
#    print(k,": ",v)
    
#post_cov = utils.collect(p_xt_update.covar.to_full_expr(),[Q+A*F_prev*A.T],'right')
#from symgp import SuperMatAdd, SuperMatMul

In [24]:
#utils.display_expr_tree(post_cov)
#for k,v in utils.getExprsAtDepth(post_cov,range(8+1)).items():
#    print(k,": ",v)

In [25]:
#print(post_cov)
#repl_dict = {Q + A*F_prev*A.T: p_xt_predict.covar}
#new_cov_1 = utils.replace(post_cov, repl_dict,debug=False).doit()
#print(new_cov_1)

In [26]:
"""post_mean = p_xt_update.mean.to_full_expr()
print(post_mean)
repl_dict = {Q + A*F_prev*A.T: p_xt_predict.covar, A*f_prev + b_t: p_xt.mean}
new_mean_1 = utils.replace(post_mean, repl_dict).doit()
print(new_mean_1)"""

'post_mean = p_xt_update.mean.to_full_expr()\nprint(post_mean)\nrepl_dict = {Q + A*F_prev*A.T: p_xt_predict.covar, A*f_prev + b_t: p_xt.mean}\nnew_mean_1 = utils.replace(post_mean, repl_dict).doit()\nprint(new_mean_1)'

In [27]:
"""from symgp import SuperMatAdd
from sympy import expand"""

'from symgp import SuperMatAdd\nfrom sympy import expand'

In [28]:
"""k = A*f_prev + b_t
sub_expr = k.args[0]
for c in k.args[1:]:
    sub_expr += c
print([post_mean.args[1].shape])
l = list(k.args) + [post_mean.args[1]+w_t]
print("l: ",l)
print(SuperMatAdd(*l))
print(SuperMatAdd(*l).match(post_mean))"""

'k = A*f_prev + b_t\nsub_expr = k.args[0]\nfor c in k.args[1:]:\n    sub_expr += c\nprint([post_mean.args[1].shape])\nl = list(k.args) + [post_mean.args[1]+w_t]\nprint("l: ",l)\nprint(SuperMatAdd(*l))\nprint(SuperMatAdd(*l).match(post_mean))'

In [29]:
"""S_t = Constant('S_{t}',R.shape[0],R.shape[1], full_expr=R + C*p_xt_predict.covar*C.T)
repl_dict = {R + C*p_xt_predict.covar*C.T: S_t}
print(repl_dict)
new_cov_2 = utils.replace(new_cov_1, repl_dict).doit()
print(new_cov_2)"""

"S_t = Constant('S_{t}',R.shape[0],R.shape[1], full_expr=R + C*p_xt_predict.covar*C.T)\nrepl_dict = {R + C*p_xt_predict.covar*C.T: S_t}\nprint(repl_dict)\nnew_cov_2 = utils.replace(new_cov_1, repl_dict).doit()\nprint(new_cov_2)"

In [30]:
"""new_mean_2 = utils.replace(new_mean_1, repl_dict).doit()
print(new_mean_2)"""

'new_mean_2 = utils.replace(new_mean_1, repl_dict).doit()\nprint(new_mean_2)'

In [31]:
"""K_t = Constant('K_{t}', x_t.shape[0], C.shape[0], full_expr=p_xt_predict.covar*C.T*S_t.I)
repl_dict = {p_xt_predict.covar*C.T*S_t.I: K_t}
print(repl_dict)
new_cov_3 = utils.replace(new_cov_2, repl_dict).doit()
print(new_cov_3)
print(utils.expand_to_fullexpr(new_cov_3))"""

"K_t = Constant('K_{t}', x_t.shape[0], C.shape[0], full_expr=p_xt_predict.covar*C.T*S_t.I)\nrepl_dict = {p_xt_predict.covar*C.T*S_t.I: K_t}\nprint(repl_dict)\nnew_cov_3 = utils.replace(new_cov_2, repl_dict).doit()\nprint(new_cov_3)\nprint(utils.expand_to_fullexpr(new_cov_3))"

In [32]:
"""r_t = Constant('r_{t}', y_t.shape[0], y_t.shape[1], full_expr=y_t - (C*p_xt.mean + d_t))
repl_dict[y_t - (C*p_xt.mean + d_t)] = r_t
new_mean_3 = utils.replace(new_mean_2, repl_dict).doit()
print(new_mean_3)"""

"r_t = Constant('r_{t}', y_t.shape[0], y_t.shape[1], full_expr=y_t - (C*p_xt.mean + d_t))\nrepl_dict[y_t - (C*p_xt.mean + d_t)] = r_t\nnew_mean_3 = utils.replace(new_mean_2, repl_dict).doit()\nprint(new_mean_3)"

In [33]:
#myexpr=SuperMatAdd(SuperMatAdd(Q,A*F_prev*A.T),-(Q+A*F_prev*A.T)*C.T*(R + C*(Q+A*F_prev*A.T)*C.T).I*C*(Q + A*F_prev*A.T))
#print(utils.collect(myexpr,[Q+A*F_prev*A.T],'left'))
"""print(utils.collect((Q+A*F_prev*A.T)*Identity(m)-(Q+A*F_prev*A.T)*C.T*(R + C*(Q+A*F_prev*A.T)*C.T).I*C*(Q + A*F_prev*A.T) + (Q+A*F_prev*A.T)*C.T*C*(Q+A*F_prev*A.T),
     [Q+A*F_prev*A.T],'right'))"""

"print(utils.collect((Q+A*F_prev*A.T)*Identity(m)-(Q+A*F_prev*A.T)*C.T*(R + C*(Q+A*F_prev*A.T)*C.T).I*C*(Q + A*F_prev*A.T) + (Q+A*F_prev*A.T)*C.T*C*(Q+A*F_prev*A.T),\n     [Q+A*F_prev*A.T],'right'))"

In [34]:
print(p_xt_update.covar.to_full_expr())

Q + A*P_{t-1|t-1}*A' + (-1)*(Q + A*P_{t-1|t-1}*A')*C'*(R + C*(Q + A*P_{t-1|t-1}*A')*C')^-1*C*(Q + A*P_{t-1|t-1}*A')


In [35]:
print(utils.getEnds(p_xt_update.covar.to_full_expr()))

[[(Q,), (A, A'), (Q + A*P_{t-1|t-1}*A', Q + A*P_{t-1|t-1}*A')], [(Q + A*P_{t-1|t-1}*A', Q + A*P_{t-1|t-1}*A'), (Q + A*P_{t-1|t-1}*A',), (0,)]]


In [36]:
simp_exprs, subs = utils.simplify(p_xt_update.covar.to_full_expr(),debug=True)

depth:  8
usedNames:  ['x_{t-1}', 'x_t', 'y_{1:t-1}', 'y_t', 'b_t', 'd_t', 'w_t', 'e_t', 'A', 'C', 'Q', 'R', 'm_{t-1|t-1}', 'P_{t-1|t-1}', 'm_{x_{t-1}|y_{1:t-1}}', 'S_{x_{t-1}|y_{1:t-1}}', 'm_{x_t|x_{t-1}}', 'S_{x_t|x_{t-1}}', 'S_{x_t,x_t}', 'S_{x_t,x_{t-1}}', 'S_{x_{t-1},x_t}', 'S_{x_{t-1},x_{t-1}}', 'm_{x_t,x_{t-1}|y_{1:t-1}}', 'S_{x_t,x_{t-1}|y_{1:t-1}}', 'm_{x_t|y_{1:t-1}}', 'S_{x_t|y_{1:t-1}}', 'm_{y_t|x_t}', 'S_{y_t|x_t}', 'S_{y_t,y_t}', 'S_{y_t,x_t}', 'S_{x_t,y_t}', 'S_{x_t,x_t}', 'm_{y_t,x_t|y_{1:t-1}}', 'S_{y_t,x_t|y_{1:t-1}}', 'S_{x_t}', 'S_{x_t,y_t}', 'S_{y_t,x_t}', 'S_{y_t}', 'm_{x_t|y_{1:t-1},y_t}', 'S_{x_t|y_{1:t-1},y_t}']
d:  8
min_expr:  Q + A*P_{t-1|t-1}*A' + (-1)*(Q + A*P_{t-1|t-1}*A')*C'*(R + C*(Q + A*P_{t-1|t-1}*A')*C')^-1*C*(Q + A*P_{t-1|t-1}*A')
exprs_by_depth:  defaultdict(<class 'list'>, {0: [Q + A*P_{t-1|t-1}*A', Q + (-1)*(Q + A*P_{t-1|t-1}*A')*C'*(R + C*(Q + A*P_{t-1|t-1}*A')*C')^-1*C*(Q + A*P_{t-1|t-1}*A'), A*P_{t-1|t-1}*A' + (-1)*(Q + A*P_{t-1|t-1}*A')*C'*(R

In [37]:
B_99 = SuperMatSymbol(Q.shape[0],Q.shape[1],'B_{99}')
d = {Q + A*F_prev*A.T: B_99}

print(utils.replace(p_xt_update.covar.to_full_expr(), d).doit())

(-1)*B_{99}*C'*(R + C*B_{99}*C')^-1*C*B_{99} + B_{99}


In [38]:
print(utils.getExprsAtDepth(utils.replace(p_xt_update.covar.to_full_expr(), d).doit(),range(8),debug=True))

depths:  [0, 1, 2, 3, 4, 5, 6, 7]
sub_expr: (-1)*B_{99}*C'*(R + C*B_{99}*C')^-1*C*B_{99} + B_{99}, level: 0
sub_expr:  (-1)*B_{99}*C'*(R + C*B_{99}*C')^-1*C*B_{99} + B_{99}
exprs_at_depth[0]: [(-1)*B_{99}*C'*(R + C*B_{99}*C')^-1*C*B_{99} + B_{99}]
sub_expr.args:  ((-1)*B_{99}*C'*(R + C*B_{99}*C')^-1*C*B_{99}, B_{99})
sub_expr: (-1)*B_{99}*C'*(R + C*B_{99}*C')^-1*C*B_{99}, level: 1
sub_expr:  (-1)*B_{99}*C'*(R + C*B_{99}*C')^-1*C*B_{99}
Matched E^{-1}
Matched E^{-1}F
Matched E^{-1}F({MatExpr})^{-1}
Matched E^{-1}F({MatAdd})^{-1}
Matched E^{-1}F(A+B)^{-1}
Matched E^{-1}F(A + GCD)^{-1} or E^{-1}F(A + (-1)*GCD)^{-1}
exprs_at_depth[1]: [B_{99}*C'*(R + C*B_{99}*C')^-1]
sub_expr.args:  (-1, B_{99}, C', (R + C*B_{99}*C')^-1, C, B_{99})
sub_expr: -1, level: 2
sub_expr: B_{99}, level: 2
sub_expr: C', level: 2
sub_expr.args:  (C,)
sub_expr: C, level: 3
sub_expr: (R + C*B_{99}*C')^-1, level: 2
sub_expr.args:  (R + C*B_{99}*C',)
sub_expr: R + C*B_{99}*C', level: 3
sub_expr:  R + C*B_{99}*C'
exprs_a

In [39]:
for s in simp_exprs:
    print("s: ", s)

for s in subs.keys():
    print("%s: %s"%(s,subs[s]))

s:  Q + (-1)*(Q + A_{0})*C'*(R + C*(Q + A_{0})*C')^-1*C*(Q + A_{0}) + A_{0}
s:  (Q + A_{0})*(I + (-1)*C'*(R + C*(Q + A_{0})*C')^-1*C*(Q + A_{0}))
s:  (I + (-1)*(Q + A_{0})*C'*(R + C*(Q + A_{0})*C')^-1*C)*(Q + A_{0})
s:  (-1)*A_{1}*C'*(R + C*A_{1}*C')^-1*C*A_{1} + A_{1}
s:  A_{1}*(I + (-1)*C'*(R + C*A_{1}*C')^-1*C*A_{1})
s:  (I + (-1)*A_{1}*C'*(R + C*A_{1}*C')^-1*C)*A_{1}
s:  (-1)*A_{2}*C*A_{1} + A_{1}
s:  (I + (-1)*A_{2}*C)*A_{1}
Q + A_{0}: A_{1}
A_{1}*C'*(R + C*A_{1}*C')^-1: A_{2}
A*P_{t-1|t-1}*A': A_{0}


In [40]:
sorted_simp_exprs = sorted(simp_exprs, key=lambda x: utils.getNumSymbols(x))
[print(s) for s in sorted_simp_exprs]

(I + (-1)*A_{2}*C)*A_{1}
(-1)*A_{2}*C*A_{1} + A_{1}
A_{1}*(I + (-1)*C'*(R + C*A_{1}*C')^-1*C*A_{1})
(I + (-1)*A_{1}*C'*(R + C*A_{1}*C')^-1*C)*A_{1}
(-1)*A_{1}*C'*(R + C*A_{1}*C')^-1*C*A_{1} + A_{1}
(Q + A_{0})*(I + (-1)*C'*(R + C*(Q + A_{0})*C')^-1*C*(Q + A_{0}))
(I + (-1)*(Q + A_{0})*C'*(R + C*(Q + A_{0})*C')^-1*C)*(Q + A_{0})
Q + (-1)*(Q + A_{0})*C'*(R + C*(Q + A_{0})*C')^-1*C*(Q + A_{0}) + A_{0}


[None, None, None, None, None, None, None, None]

In [41]:
K_t = Constant('K_t',x_t.shape[0],C.shape[0],full_expr=)

SyntaxError: invalid syntax (<ipython-input-41-635b06e76bb5>, line 1)

In [ ]:
simp_exprs_2, subs_2 = utils.simplify(p_xt_update.mean.to_full_expr(),debug=True)

In [ ]:
print(p_xt_update.mean.to_full_expr())

In [ ]:
for s in simp_exprs_2:
    print("s: ", s)

for s in subs_2.keys():
    print("%s: %s"%(s,subs_2[s]))

In [ ]:
keys=list(subs.keys())
print(simp_exprs[3])
#print(s[3], subs[keys[0]])
print(utils.collect(simp_exprs[3],subs[keys[0]],'left'))

In [ ]:
print(utils.expand_matexpr((A*Q + 2*Identity(m))*A,debug=False).doit().as_coeff_mmul())
print(utils.collect(A*Q*A + 2*A,[A],'right'))

In [ ]:
from sympy import srepr

In [ ]:
print((2*A).as_coeff_mmul())
print(A.I.as_coeff_mmul())

In [ ]:
print(A+Q == Q+A)

In [ ]:
print(srepr(Identity(m)*2*A))
print((2*Identity(m)*A))

In [ ]:
print(p_xt_update.covar.to_full_expr())

In [ ]:
print(utils.replace(p_xt_update.covar.to_full_expr(), {A*F_prev*A.T: SuperMatSymbol(m,m,'B_{99}')},debug=False))

In [ ]:
from sympy import MatrixSymbol, MatMul
from symgp import SuperMatMul

In [ ]:
print()

In [ ]:
print(SuperMatMul(*(2*Identity(n)).args,A_76).doit())

In [ ]:
A_76 = MatrixSymbol('A_{76}',n,n)
print(MatMul(MatMul(2,Identity(n)),A_76).doit())
#print((2*Identity(n)*A_76).args)